In [2]:
import pandas as pd 
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go 

In [5]:
print('loading titanic dataset...')
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)
print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns\n")


loading titanic dataset...
Dataset loaded: 891 rows, 12 columns



In [6]:
print("="*60)
print("1. DATA OVERVIEW")
print("="*60)

1. DATA OVERVIEW


In [10]:
print("First 5 rows:")
print(df.head(), "\n")

First 5 rows:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN

In [14]:
print("Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing': missing, '%': missing_pct.round(2)})
missing_df = missing_df[missing_df['Missing'] > 0]
print(missing_df, "\n")

Missing Values:
          Missing      %
Age           177  19.87
Cabin         687  77.10
Embarked        2   0.22 



In [18]:
print("Data Types:")
print(df.dtypes, "\n")

Data Types:
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object 



In [19]:
print("="*60)
print("2. DATA CLEANING & FEATURE ENGINEERING")
print("="*60)

2. DATA CLEANING & FEATURE ENGINEERING


In [20]:
df_clean = df.copy()

In [21]:
df_clean['Age'] = df_clean.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
df_clean['Age'].fillna(df_clean['Age'].median(), inplace=True)

C:\Users\Anik\AppData\Local\Temp\ipykernel_9460\2453202509.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Age'].fillna(df_clean['Age'].median(), inplace=True)


In [22]:
df_clean['Embarked'].fillna(df_clean['Embarked'].mode()[0], inplace=True)

C:\Users\Anik\AppData\Local\Temp\ipykernel_9460\872357374.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Embarked'].fillna(df_clean['Embarked'].mode()[0], inplace=True)


In [23]:
df_clean['HasCabin'] = df_clean['Cabin'].notna().astype(int)
df_clean['Deck'] = df_clean['Cabin'].str[0].fillna('Unknown')

In [24]:
df_clean['Fare'].fillna(df_clean['Fare'].median(), inplace=True)

C:\Users\Anik\AppData\Local\Temp\ipykernel_9460\2316612605.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Fare'].fillna(df_clean['Fare'].median(), inplace=True)


In [25]:
df_clean['FamilySize'] = df_clean['SibSp'] + df_clean['Parch'] + 1
df_clean['IsAlone'] = (df_clean['FamilySize'] == 1).astype(int)

In [26]:
df_clean['Title'] = df_clean['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_clean['Title'] = df_clean['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 
                                             'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df_clean['Title'] = df_clean['Title'].replace('Mlle', 'Miss').replace('Ms', 'Miss').replace('Mme', 'Mrs')

print("Feature engineering completed!\n")

Feature engineering completed!



<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Anik\AppData\Local\Temp\ipykernel_9460\3046658854.py:1: SyntaxWarning: invalid escape sequence '\.'
  df_clean['Title'] = df_clean['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [27]:
fig1 = px.pie(
    df_clean, names='Survived',
    title='Overall Survival Rate',
    color_discrete_sequence=['#e74c3c', '#27ae60'],
    labels={'0': 'Died', '1': 'Survived'}
)
fig1.update_traces(textposition='inside', textinfo='percent+label')
fig1.show()

In [28]:
fig2 = px.histogram(
    df_clean, x='Age', color='Survived', nbins=30,
    title='Age Distribution by Survival',
    marginal='box',
    color_discrete_sequence=['#e74c3c', '#27ae60'],
    labels={'0': 'Died', '1': 'Survived'}
)
fig2.update_layout(barmode='stack')
fig2.show()

In [29]:
fig3 = px.histogram(
    df_clean, x='Fare', nbins=50,
    title='Fare Distribution',
    color_discrete_sequence=['#3498db']
)
fig3.show()

In [30]:
fig4 = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Survival by Class', 'Survival by Sex', 'Survival by Embarked'),
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]]
)

pclass_rate = df_clean.groupby('Pclass')['Survived'].mean()
sex_rate = df_clean.groupby('Sex')['Survived'].mean()
emb_rate = df_clean.groupby('Embarked')['Survived'].mean()

fig4.add_trace(go.Bar(x=pclass_rate.index, y=pclass_rate.values, name='Pclass'), 1, 1)
fig4.add_trace(go.Bar(x=sex_rate.index, y=sex_rate.values, name='Sex'), 1, 2)
fig4.add_trace(go.Bar(x=emb_rate.index, y=emb_rate.values, name='Embarked'), 1, 3)

fig4.update_layout(title_text="Survival Rates by Key Features", showlegend=False)
fig4.show()

In [31]:
fig5 = px.scatter(
    df_clean, x='Age', y='Fare', color='Survived', size='Pclass',
    hover_data=['Sex', 'Embarked', 'Title'],
    title='Age vs Fare Colored by Survival',
    color_discrete_sequence=['#e74c3c', '#27ae60']
)
fig5.show()

In [32]:
num_cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'HasCabin']
corr = df_clean[num_cols].corr()

fig6 = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=corr.columns,
    y=corr.columns,
    colorscale='RdBu',
    zmid=0,
    text=corr.values,
    texttemplate="%{text:.2f}",
    textfont={"size": 10}
))
fig6.update_layout(title='Correlation Matrix', height=600)
fig6.show()

In [33]:
pivot = df_clean.pivot_table('Survived', index='Pclass', columns='Sex', aggfunc='mean')

fig7 = px.imshow(
    pivot.values,
    x=pivot.columns,
    y=pivot.index,
    title='Survival Rate by Pclass & Sex',
    labels=dict(x="Sex", y="Pclass", color="Survival Rate"),
    color_continuous_scale='Viridis',
    text_auto=True
)
fig7.show()

In [34]:
print("="*60)
print("KEY INSIGHTS")
print("="*60)
insights = """
1. Survival Rate: ~38.4% survived
2. 1st Class: 63% survived | 3rd Class: 24%
3. Females: 74% survived | Males: 19%
4. Children (<10) had higher survival
5. Passengers with Cabin: 66% survived
6. Small families (2–4) survived more than alone or large
7. Titles: 'Mrs', 'Miss' > 'Mr' > 'Rare'
"""
print(insights)

KEY INSIGHTS

1. Survival Rate: ~38.4% survived
2. 1st Class: 63% survived | 3rd Class: 24%
3. Females: 74% survived | Males: 19%
4. Children (<10) had higher survival
5. Passengers with Cabin: 66% survived
6. Small families (2–4) survived more than alone or large
7. Titles: 'Mrs', 'Miss' > 'Mr' > 'Rare'



In [35]:
model_df = df_clean.copy()
model_df = pd.get_dummies(model_df, columns=['Sex', 'Embarked', 'Title', 'Deck'], drop_first=True)

features = ['Pclass', 'Age', 'Fare', 'FamilySize', 'IsAlone', 'HasCabin',
            'Sex_male', 'Embarked_Q', 'Embarked_S',
            'Title_Miss', 'Title_Mrs', 'Title_Rare']

X = model_df[features]
y = model_df['Survived']

print(f"Final feature matrix: {X.shape}")
print("Ready for machine learning!\n")

print("EDA COMPLETE! All plots are interactive.")

Final feature matrix: (891, 12)
Ready for machine learning!

EDA COMPLETE! All plots are interactive.
